In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes


img = read_image('/Users/viktor/polsl/bachelor_project/DatasetTraffic/dataset/sec1/sec1_frame_004492.png'
bbox = [[0.019802, 0.015259, 0.396367*1920, 0.304398*1080],[0.206458, 0.373509, 0.019802, 0.015259]]
bbox = torch.tensor(bbox, dtype=torch.int)
top_left_x = center_x - width / 2
top_left_y = center_y - height / 2
bottom_right_x = center_x + width / 2
bottom_right_y = center_y + height / 2
adjusted_box = [top_left_x, top_left_y, bottom_right_x, bottom_right_y]

print(bbox)
print(bbox.size())

img=draw_bounding_boxes(img, bbox, width=3, colors=[(255,0,0),(0,255,0)])
img = torchvision.transforms.ToPILImage()(img)
img.show()




In [ ]:
# %%time
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    print("MPS found")
    device = torch.device("mps")

In [ ]:
class TrafficDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, transforms=None):
        self.root_path = root_path
        self.transforms = transforms

        self.frames, self.labels = self.__get_frames_labels(self.root_path)
    
    def __get_frames_labels(self, root_path):
        frames = []
        labels = []
        for _, _, files in os.walk(self.root_path):
            sorted_files = sorted(files)
            for id, file in enumerate(sorted_files[1::2]):
                labels.append(os.path.join(sec1, file))
                frames.append(os.path.join(sec1, sorted_files[id*2]))
        return tuple(frames), tuple(labels)

    def __getitem__(self, idx):
        frame = self.frames[idx]
        label = self.labels[idx]

        frame = read_image(frame)
        with open(label) as f:
            label = f.readlines()
        num_objs = len(label)
        



current_path = os.path.dirname(os.path.abspath("__file__"))
dataset_path = os.path.join(current_path, "..", "DatasetTraffic", "dataset", "sec1")

obj = TrafficDataset(dataset_path)

obj[-1]
